In [166]:
import pandas as pd
from stockstats import *
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [169]:
bc = StockDataFrame.retype(pd.read_csv('coinbase.csv'))

In [170]:
bc

,currency,close,open (usd),high,low
date,,,,,
2013-10-01,BTC,123.654990,124.304660,124.751660,122.563490
2013-10-02,BTC,125.455000,123.654990,125.758500,123.633830
2013-10-03,BTC,108.584830,125.455000,125.665660,83.328330
2013-10-04,BTC,118.674660,108.584830,118.675000,107.058160
2013-10-05,BTC,121.338660,118.674660,121.936330,118.005660
...,...,...,...,...,...
2021-02-24,BTC,48745.432984,48923.035081,51413.613557,47032.524262
2021-02-25,BTC,48291.412083,49746.651631,52076.317723,47501.674798
2021-02-26,BTC,45752.114919,47135.551728,48453.738635,44279.449538


In [171]:
bc['rsi_7']

date
2013-10-01           NaN
2013-10-02    100.000000
2013-10-03      8.379203
2013-10-04     44.109832
2013-10-05     50.103761
                 ...    
2021-02-24     44.976954
2021-02-25     43.539693
2021-02-26     36.027535
2021-02-27     40.245589
2021-02-28     35.493731
Name: rsi_7, Length: 2698, dtype: float64

In [172]:
bc['close_-1_r']

date
2013-10-01          NaN
2013-10-02     1.455671
2013-10-03   -13.447188
2013-10-04     9.292118
2013-10-05     2.244793
                ...    
2021-02-24     1.188543
2021-02-25    -0.931412
2021-02-26    -5.258279
2021-02-27     1.946339
2021-02-28    -3.322712
Name: close_-1_r, Length: 2698, dtype: float64

In [161]:
new_bc = bc.drop(columns=['open (usd)', 'high','low','close_-1_s','close_-1_d','rs_7'])

In [173]:
new_bc

,currency,close,rsi_7,close_-1_r,next_close
date,,,,,
2013-10-01,BTC,123.654990,NaN,NaN,125.455000
2013-10-02,BTC,125.455000,100.000000,1.455671,108.584830
2013-10-03,BTC,108.584830,8.379203,-13.447188,118.674660
2013-10-04,BTC,118.674660,44.109832,9.292118,121.338660
2013-10-05,BTC,121.338660,50.103761,2.244793,120.655330
...,...,...,...,...,...
2021-02-24,BTC,48745.432984,44.976954,1.188543,48291.412083
2021-02-25,BTC,48291.412083,43.539693,-0.931412,45752.114919
2021-02-26,BTC,45752.114919,36.027535,-5.258279,46642.606077


In [174]:
new_bc['next_close'] = new_bc['close'].shift(-1)

In [175]:
dataset = new_bc.tail(600)

In [179]:
dataset = dataset.dropna()

In [180]:
X = dataset[['close', 'rsi_7', 'close_-1_r']]
y = dataset['next_close']

In [181]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [182]:
from sklearn.linear_model import LinearRegression
regressor = LinearRegression()
regressor.fit(X_train, y_train)

LinearRegression()

In [183]:
coeff_df = pd.DataFrame(regressor.coef_, X.columns, columns=['Coefficient'])
coeff_df

,Coefficient
close,1.007853
rsi_7,3.622084
close_-1_r,-18.269533


In [184]:
y_pred = regressor.predict(X_test)

In [185]:
df = pd.DataFrame({'Actual': y_test, 'Predicted': y_pred})

In [186]:
close_only = new_bc['close']

In [187]:
df = df.join(close_only)

In [213]:
df['downward_prediction'] =((df["Predicted"] < df["close"]))
df['correct_downward_prediction'] =((df["Predicted"] < df["close"]) & (df["Actual"] < df["close"]) )
df['upward_prediction'] =((df["Predicted"] > df["close"]))
df['correct_upward_prediction'] =((df["Predicted"] > df["close"]) & (df["Actual"] > df["close"]) )

In [214]:
df.shape

(120, 7)

In [215]:
pd.options.display.max_rows = None
df

,Actual,Predicted,close,downward_prediction,correct_downward_prediction,upward_prediction,correct_upward_prediction
date,,,,,,,
2019-11-14,8500.643558,8635.808509,8656.970922,True,True,False,False
2019-08-26,10122.196533,10277.523732,10312.215040,True,True,False,False
2020-11-08,15283.780139,15555.586985,15500.334255,False,False,True,False
2020-08-22,11666.410122,11683.287849,11674.624354,False,False,True,False
2020-02-20,9670.858654,9635.641375,9631.484946,False,False,True,True
2020-06-13,9351.953728,9455.415699,9458.659754,True,True,False,False
2020-03-05,9126.636822,9028.358779,9078.578777,True,False,False,False
2020-12-14,19443.476353,19372.263517,19251.224005,False,False,True,True
2021-02-22,48172.877477,54694.123379,54181.914649,False,False,True,False


In [216]:
downward = df[df['downward_prediction']] == True
correct_downward = df[df['correct_downward_prediction']] == True

In [217]:
print("Downward Predictions: " + str(downward.shape[0]))
print("Correct Downward Predictions: " + str(correct_downward.shape[0]))

Downward Predictions: 36
Correct Downward Predictions: 13


In [218]:
upward = df[df['upward_prediction']] == True
correct_upward = df[df['correct_upward_prediction']] == True
print("Upward Predictions: " + str(upward.shape[0]))
print("Correct Upward Predictions: " + str(correct_upward.shape[0]))

Upward Predictions: 84
Correct Upward Predictions: 47
